In [4]:
import json
from bs4 import Tag
from tqdm import tqdm
import shangri_latibet_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [5]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = shangri_latibet_utils.scrape_shangri_latibet_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_scrape_shangri_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"scrape_shangri_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [10]:
%%time
# Example usage
directory_path = './data/links/'
# directory_path = './data/test_link/'
get_json_files(directory_path)

Total page in shangri_latibet_ALL_link_ས་གནས་གསར་འགྱུར།.json: 150
page key name: Page ས་གནས་གསར་འགྱུར། 150
Page key code: Page ས་གནས་གསར་འགྱུར། 
Total page in shangri_latibet_ALL_link_རྒྱལ་ཕྱི་ནང་གསར་འགྱུར།.json: 29
page key name: Page རྒྱལ་ཕྱི་ནང་གསར་འགྱུར། 29
Page key code: Page རྒྱལ་ཕྱི་ནང་གསར་འགྱུར། 
Total page in shangri_latibet_ALL_link_ཏང་གི་ཚོགས་ཆེན་བཅོ་བརྒྱད་པ།.json: 2
page key name: Page ཏང་གི་ཚོགས་ཆེན་བཅོ་བརྒྱད་པ། 2
Page key code: Page ཏང་གི་ཚོགས་ཆེན་བཅོ་བརྒྱད་པ། 
Total page in shangri_latibet_ALL_link_བོད་ཁུལ་གསར་འགྱུར།.json: 74
page key name: Page བོད་ཁུལ་གསར་འགྱུར། 74
Page key code: Page བོད་ཁུལ་གསར་འགྱུར། 
Total page in shangri_latibet_ALL_link_ཚོགས་ཆེན་གཉིས།.json: 2
page key name: Page ཚོགས་ཆེན་གཉིས། 2
Page key code: Page ཚོགས་ཆེན་གཉིས། 
Total page in shangri_latibet_ALL_link_གལ་ཆེའི་མཛད་སྒོ།.json: 2
page key name: Page གལ་ཆེའི་མཛད་སྒོ། 2
Page key code: Page གལ་ཆེའི་མཛད་སྒོ། 
Total page in shangri_latibet_ALL_link_དཔལ་འབྱོར་འགྱུར་སྟངས།.json: 4
page key name: Page དཔལ་འབ

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]


  0%|          | 0/33 [00:00<?, ?it/s]



  0%|          | 0/40 [00:00<?, ?it/s]




  0%|          | 0/38 [00:00<?, ?it/s]





  0%|          | 0/27 [00:00<?, ?it/s]






  0%|          | 0/37 [00:00<?, ?it/s]







  0%|          | 0/568 [00:00<?, ?it/s]








  0%|          | 0/1468 [00:00<?, ?it/s]









  0%|          | 0/3000 [00:00<?, ?it/s]









  0%|          | 1/3000 [00:01<1:29:10,  1.78s/it]








  0%|          | 1/1468 [00:01<45:35,  1.86s/it]




  5%|▍         | 1/22 [00:04<00:55,  2.65s/it]

  1%|▏         | 1/80 [00:04<03:29,  2.65s/it]







  0%|          | 1/568 [00:04<23:36,  2.50s/it]






  3%|▎         | 1/37 [00:04<01:36,  2.67s/it]





  4%|▎         | 1/27 [00:04<01:09,  2.69s/it]









  0%|          | 2/3000 [00:04<1:08:57,  1.38s/it]



  2%|▎         | 1/40 [00:04<02:04,  3.18s/it]








  0%|          | 2/1468 [00:04<37:43,  1.54s/it]




  9%|▉       

Total Failure in the ཏང་གི་ཚོགས་ཆེན་བཅོ་བརྒྱད་པ། article: 0
scrape_shangri_ALL_content_ཏང་གི་ཚོགས་ཆེན་བཅོ་བརྒྱད་པ།.json
Successfully saved: scrape_shangri_ALL_content_ཏང་གི་ཚོགས་ཆེན་བཅོ་བརྒྱད་པ།.json
Total page in shangri_latibet_ALL_link_སློབ་གསོ།.json: 12
page key name: Page སློབ་གསོ། 12
Page key code: Page སློབ་གསོ། 


  0%|          | 0/232 [00:00<?, ?it/s]



 70%|███████   | 28/40 [00:10<00:02,  4.65it/s]








  2%|▏         | 29/1468 [00:10<04:53,  4.90it/s]









  1%|          | 29/3000 [00:10<10:45,  4.60it/s]





100%|██████████| 27/27 [00:11<00:00,  2.91it/s]






 78%|███████▊  | 29/37 [00:11<00:02,  3.19it/s]



 75%|███████▌  | 30/40 [00:11<00:02,  4.21it/s]









 59%|█████▉    | 29/49 [00:11<00:05,  3.40it/s]

 36%|███▋      | 29/80 [00:11<00:14,  3.43it/s]




 79%|███████▉  | 30/38 [00:11<00:02,  3.91it/s]






 61%|██████    | 30/49 [00:11<00:05,  3.63it/s]

100%|██████████| 27/27 [00:11<00:00,  2.33it/s]


Total Failure in the གལ་ཆེའི་མཛད་སྒོ། article: 0
scrape_shangri_ALL_content_གལ་ཆེའི་མཛད་སྒོ།.json
Successfully saved: scrape_shangri_ALL_content_གལ་ཆེའི་མཛད་སྒོ།.json
Total page in shangri_latibet_ALL_link_རྩོམ་རིག.json: 6
page key name: Page རྩོམ་རིག 6
Page key code: Page རྩོམ་རིག 


...

Total Failure in the བོད་ཁུལ་གསར་འགྱུར། article: 3
scrape_shangri_ALL_content_བོད་ཁུལ་གསར་འགྱུར།.json
Successfully saved: scrape_shangri_ALL_content_བོད་ཁུལ་གསར་འགྱུར།.json












 49%|████▉     | 1467/3000 [07:18<07:29,  3.41it/s]









 49%|████▉     | 1469/3000 [07:19<07:29,  3.40it/s]









 49%|████▉     | 1470/3000 [07:19<09:08,  2.79it/s]









 49%|████▉     | 1471/3000 [07:20<08:27,  3.02it/s]









 49%|████▉     | 1472/3000 [07:20<08:50,  2.88it/s]









 49%|████▉     | 1473/3000 [07:20<07:12,  3.53it/s]









 49%|████▉     | 1475/3000 [07:20<04:47,  5.30it/s]









 49%|████▉     | 1476/3000 [07:21<07:13,  3.52it/s]









 49%|████▉     | 1477/3000 [07:21<07:09,  3.54it/s]









 49%|████▉     | 1478/3000 [07:21<06:42,  3.78it/s]









 49%|████▉     | 1480/3000 [07:23<11:00,  2.30it/s]









 49%|████▉     | 1481/3000 [07:23<09:55,  2.55it/s]









 49%|████▉     | 1482/3000 [07:23<09:48,  2.58it/s]









 49%|████▉     | 1484/3000 [07:24<07:57,  3.18it/s]









 50%|████▉     | 1485/3000 [07:24<07:05,  3.56it/s]









 50%|████▉     | 1487/3000 [07:24<04:51,  5.20it/s]









 50%|████▉    

Total Failure in the ས་གནས་གསར་འགྱུར། article: 10
scrape_shangri_ALL_content_ས་གནས་གསར་འགྱུར།.json





100%|██████████| 33/33 [14:43<00:00, 26.78s/it]  

Successfully saved: scrape_shangri_ALL_content_ས་གནས་གསར་འགྱུར།.json
Processed 33 files
CPU times: user 3min 52s, sys: 13.7 s, total: 4min 5s
Wall time: 14min 43s


In [ ]:
# 14:44

In [7]:
pwd()

'/workspace/web_scrab/new_news_Articles/shangri-latibet'

In [11]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files

NameError: name 'directory' is not defined